In [1]:
# import numpy as np

In [2]:
# Y = np.array([[1, 1, 0, 0], [1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 1]])
# C = np.random.normal(0, 1, (4, 2))
# X = np.random.normal(0, 1, (2, 4))

In [3]:
# def sigmoid(z):
#     return 1/(1 + np.exp(-z))

In [4]:
# gamma = 0.01
# for i in range(1000):
#     X = X + gamma * C.T @ (Y * (1 - sigmoid(C @ X)) - (1 - Y) * sigmoid(C @ X))
#     C = C + gamma * (Y * (1 - sigmoid(C @ X)) - (1 - Y) * sigmoid(C @ X)) @ X.T
# print(np.round(sigmoid(C @ X)))
# print(C)
# print(X)

In [5]:
import sys
sys.path.append('../Library')

from Modules.BernoulliDF import BernoulliDF as DictionaryFilter

2024-08-03 11:42:56.384304: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-03 11:42:56.406310: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-03 11:42:56.406321: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-03 11:42:56.407125: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-03 11:42:56.411354: I tensorflow/core/platform/cpu_feature_guar

In [6]:
import tensorflow as tf
import tensorflow_ranking as tfr
import pandas as pd
import numpy as np

In [7]:
# Read the gowalla dataset
train_data = pd.read_csv('../Data/Binary/gowalla/train.csv')
test_data = pd.read_csv('../Data/Binary/gowalla/test.csv')

In [8]:
# Print the shape of the data
print(train_data.shape)
print(test_data.shape)

(810128, 3)
(217242, 3)


In [9]:
NUM_MOVIES = train_data['Movie Index'].max() + 1
NUM_USERS = train_data['User Index'].max() + 1
NUM_FACTORS = 16
BATCH_SIZE = 256
GAMMA_C = 0.001
GAMMA_X = 0.001
SIGMA_C = 1.0

In [10]:
# Convert train data to sparse tensor
train_sparse_tensor = tf.sparse.SparseTensor(
    indices=train_data[['Movie Index', 'User Index']].values,
    values=train_data['Rating'].values,
    dense_shape=[NUM_MOVIES, NUM_USERS]
)
train_sparse_tensor = tf.sparse.reorder(train_sparse_tensor)

# Convert test data to sparse tensor
test_sparse_tensor = tf.sparse.SparseTensor(
    indices=test_data[['Movie Index', 'User Index']].values,
    values=test_data['Rating'].values,
    dense_shape=[NUM_MOVIES, NUM_USERS]
)
test_sparse_tensor = tf.sparse.reorder(test_sparse_tensor)

# Create dataset
def data_generator():
    train_slices = tf.sparse.split(sp_input=train_sparse_tensor, num_split=NUM_USERS // BATCH_SIZE, axis=1)
    test_slices = tf.sparse.split(sp_input=test_sparse_tensor, num_split=NUM_USERS // BATCH_SIZE, axis=1)
    for i in range(NUM_USERS // BATCH_SIZE):
        yield (tf.sparse.to_dense(train_slices[i]), tf.sparse.to_dense(test_slices[i]))

dataset = tf.data.Dataset.from_generator(
    data_generator, 
    output_signature=(
        tf.TensorSpec(shape=[NUM_MOVIES, None], dtype=tf.float32),
        tf.TensorSpec(shape=[NUM_MOVIES, None], dtype=tf.float32)
    )
)

2024-08-03 11:42:58.354718: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-03 11:42:58.421531: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-03 11:42:58.421597: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-03 11:42:58.423082: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-03 11:42:58.423117: I external/local_xla/xla/stream_executor

In [11]:
# Create the model
model = DictionaryFilter(NUM_MOVIES, NUM_USERS, NUM_FACTORS, GAMMA_X, GAMMA_C, SIGMA_C)
recall = tf.keras.metrics.Recall(top_k=20)
ndcg = tfr.keras.metrics.NDCGMetric(topn=20)

# Train the model
for epoch in range(100):
    print("Epoch:", epoch)

    k = tf.Variable(0, trainable=False)
    for train_batch, test_batch in dataset:
        Y_hat = model(train_batch, k)
        k.assign_add(train_batch.shape[1])

        # Compute the metrics
        Y_test = tf.multiply(Y_hat, tf.subtract(1.0, train_batch))
        recall.update_state(tf.transpose(Y_test), tf.transpose(test_batch))
        ndcg.update_state(tf.transpose(Y_test), tf.transpose(test_batch))

    # Print the metrics
    print("Average elements:", tf.reduce_mean(model.C), tf.reduce_mean(model.X))
    print("Recall:", recall.result().numpy())
    print("NDCG:", ndcg.result().numpy())
    recall.reset_states()
    ndcg.reset_states()

Epoch: 0


2024-08-03 11:42:59.131454: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Average elements: tf.Tensor(0.060521163, shape=(), dtype=float32) tf.Tensor(-1.3633932, shape=(), dtype=float32)
Recall: 0.00048815127
NDCG: 0.014020845
Epoch: 1
Average elements: tf.Tensor(0.050806653, shape=(), dtype=float32) tf.Tensor(-1.3824214, shape=(), dtype=float32)
Recall: 0.00048810363
NDCG: 0.008520123
Epoch: 2
Average elements: tf.Tensor(0.045536887, shape=(), dtype=float32) tf.Tensor(-1.3827436, shape=(), dtype=float32)
Recall: 0.00048802793
NDCG: 0.011187145
Epoch: 3
Average elements: tf.Tensor(0.04213629, shape=(), dtype=float32) tf.Tensor(-1.3830631, shape=(), dtype=float32)
Recall: 0.0004880515
NDCG: 0.014359032
Epoch: 4
Average elements: tf.Tensor(0.039687857, shape=(), dtype=float32) tf.Tensor(-1.3833587, shape=(), dtype=float32)
Recall: 0.0004880194
NDCG: 0.017739631
Epoch: 5
Average elements: tf.Tensor(0.03789368, shape=(), dtype=float32) tf.Tensor(-1.3836371, shape=(), dtype=float32)
Recall: 0.00048800764
NDCG: 0.020715412
Epoch: 6
Average elements: tf.Tensor(0.03

KeyboardInterrupt: 